*Author: Daniel Puente*   

<img src="https://cdn-icons-png.flaticon.com/512/2977/2977956.png" width="100" height="100" float ="right">   

In this notebook the dataset will be cleaned to remove all columns with null values and rows that contribute noise.    
The aim is to prepare the dataset for a deeper analysis in the next notebook `02_eda`. 
For this purpose, the following index will be used: 

- <a href='#1'><ins>1. Loading of Libraries and Data<ins></a>
- <a href='#2'><ins>2. Null analysis<ins></a>
- <a href='#3'><ins>3. Save data<ins> </a>

### <a id='1'>1. Loading of Libraries and Data</a>
----

In [1]:
import pandas as pd
import numpy as np

path = "../data/"
output_path = "../data/cleaning_generated_data/"

df = pd.read_csv(path + "application_data.csv")
info_df = pd.read_csv(path + "columns_description.csv", encoding='cp1252', index_col=0)
previous_df = pd.read_csv(path + "previous_application.csv")

### <a id='2'>2. Null analysis</a>
---

There are plenty of columns in the dataset and most of them have null values. The columns with null values will be removed.

In [5]:
null_values = np.array(sorted(list(df.isna().sum()/len(df)))[::-1])
print("Number of columns with more than 0% null values:", len(null_values[null_values > 0]))
print("Number of columns with more than 10% null values:", len(null_values[null_values > 0.1]))

Number of columns with more than 0% null values: 67
Number of columns with more than 10% null values: 57


Columns with more than 20% of null values are removed and then perform a drop of the rows with null values.   
Since there is a lot of data it is not a bad option to skip data in this way.  

In [6]:
null_column_values = df.isna().sum()/len(df)
null_column = null_column_values[null_column_values > 0.2].index

df_new = df.drop(null_column, axis=1)
print(df_new.isna().sum().sort_values(ascending=False))

df_new.dropna(inplace=True)

EXT_SOURCE_3                   60965
AMT_REQ_CREDIT_BUREAU_YEAR     41519
AMT_REQ_CREDIT_BUREAU_QRT      41519
AMT_REQ_CREDIT_BUREAU_MON      41519
AMT_REQ_CREDIT_BUREAU_WEEK     41519
                               ...  
REG_REGION_NOT_LIVE_REGION         0
REG_REGION_NOT_WORK_REGION         0
LIVE_REGION_NOT_WORK_REGION        0
REG_CITY_NOT_LIVE_CITY             0
REG_CITY_NOT_WORK_CITY             0
Length: 72, dtype: int64


A look is taken at the categorical data we can use to generate the graph.

In [7]:
non_numeric_columns = df_new.select_dtypes(exclude = np.number).columns
df_new[non_numeric_columns].head()

,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,WEEKDAY_APPR_PROCESS_START,ORGANIZATION_TYPE
0,Cash loans,M,N,Y,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,WEDNESDAY,Business Entity Type 3
2,Revolving loans,M,Y,Y,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,MONDAY,Government
5,Cash loans,M,N,Y,"Spouse, partner",State servant,Secondary / secondary special,Married,House / apartment,WEDNESDAY,Other
6,Cash loans,F,Y,Y,Unaccompanied,Commercial associate,Higher education,Married,House / apartment,SUNDAY,Business Entity Type 3
7,Cash loans,M,Y,Y,Unaccompanied,State servant,Higher education,Married,House / apartment,MONDAY,Other


Checking the distribution of the target variable. The dataset is unbalanced.

In [8]:
print(df_new.TARGET.value_counts()/len(df_new))
print("DF unique values: ", df_new.SK_ID_CURR.nunique())
print("Previous DF unique values: ", previous_df.SK_ID_CURR.nunique())

0    0.922089
1    0.077911
Name: TARGET, dtype: float64
DF unique values:  244280
Previous DF unique values:  338857


From the data of the previous application, are selected those that match the data of the current application 

In [9]:
df_new_previous = previous_df[previous_df.SK_ID_CURR.isin(df_new.SK_ID_CURR)]

The same procedure as with the previous dataset is followed. This way there are no null values. 

In [10]:
null_values_join = (df_new_previous.isna().sum()/len(df_new_previous)).sort_values(ascending=False)
threshold = 0.2

null_column_values_join = null_values_join[null_values_join > threshold].index

df_new_previous = df_new_previous.drop(null_column_values_join, axis=1)
df_new_previous.dropna(inplace=True)
df_new_previous.sort_values(by=['SK_ID_CURR', 'DAYS_DECISION'], inplace=True)

The example of user `100008` is given

In [11]:
display(df_new[df_new['SK_ID_CURR']==100008])
display(df_new_previous[df_new_previous['SK_ID_CURR']==100008])

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
5,100008,0,Cash loans,M,N,Y,0,99000.0,490495.5,27517.5,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,1.0


,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_APPLICATION,AMT_CREDIT,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NFLAG_LAST_APPL_IN_DAY,NAME_CASH_LOAN_PURPOSE,...,CODE_REJECT_REASON,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,SELLERPLACE_AREA,NAME_SELLER_INDUSTRY,NAME_YIELD_GROUP,PRODUCT_COMBINATION
604799,1186888,100008,Consumer loans,44455.5,39955.5,MONDAY,13,Y,1,XAP,...,XAP,New,Mobile,POS,XNA,Country-wide,50,Connectivity,low_normal,POS mobile with interest
906992,1907290,100008,Consumer loans,121455.0,109309.5,MONDAY,9,Y,1,XAP,...,XAP,Repeater,Gardening,POS,XNA,Regional / Local,96,Consumer electronics,low_normal,POS household with interest
800095,2613879,100008,Cash loans,450000.0,501975.0,MONDAY,10,Y,1,XNA,...,XAP,Refreshed,XNA,Cash,x-sell,Country-wide,110,Consumer electronics,middle,Cash X-Sell: middle
320227,2218188,100008,Consumer loans,162598.5,162598.5,THURSDAY,10,Y,1,XAP,...,XAP,Repeater,Consumer Electronics,POS,XNA,Country-wide,110,Consumer electronics,low_normal,POS household with interest
600145,1757431,100008,Cash loans,0.0,0.0,FRIDAY,18,Y,1,XNA,...,XAP,Repeater,XNA,XNA,XNA,Credit and cash offices,-1,XNA,XNA,Cash


Not all the columns of the `previous_df` are in the `current_df`.

In [25]:
columns_array = np.array(df_new_previous.columns) 
joined_columns = columns_array[~np.isin(columns_array, df_new.columns)]

print('Columns that match:', len(joined_columns))
print('Number of total columns:', len(columns_array))

Columns that match: 18
Number of total columns: 23


New info dataset is generated

In [12]:
new_info_df = info_df[ (info_df['Row'].isin(list(df_new.columns) + list(df_new_previous.columns)))]

Some valuable information is obtained

In [28]:
print("The shape of the current dataframe is: ", df_new.shape)
print("The unique values of the current dataframe are: ", df_new.SK_ID_CURR.nunique(), "\n")

print("The shape of the previous dataframe is: ", df_new_previous.shape)
print("The unique values of the previous dataframe are: ", df_new_previous.SK_ID_CURR.nunique())

The shape of the current dataframe is:  (244280, 72)
The unique values of the current dataframe are:  244280 

The shape of the previous dataframe is:  (1164707, 23)
The unique values of the previous dataframe are:  232196


Fraud transactions are obtained to analyse the `previous_df`

In [29]:
id_fraud = df_new[df_new['TARGET'] == 1]['SK_ID_CURR'].values
df_previous_fraud, df_previous_no_fraud = df_new_previous[df_new_previous['SK_ID_CURR'].isin(id_fraud)], df_new_previous[~df_new_previous['SK_ID_CURR'].isin(id_fraud)]

print("The shape of the previous dataframe with fraud is: ", df_previous_fraud.shape)
print("The unique values of the previous dataframe with fraud are: ", df_previous_fraud.SK_ID_CURR.nunique(), "\n")

print("The shape of the previous dataframe without fraud is: ", df_previous_no_fraud.shape)
print("The unique values of the previous dataframe without fraud are: ", df_previous_no_fraud.SK_ID_CURR.nunique())

The shape of the previous dataframe with fraud is:  (98470, 23)
The unique values of the previous dataframe with fraud are:  18296 

The shape of the previous dataframe without fraud is:  (1066237, 23)
The unique values of the previous dataframe without fraud are:  213900


### <a id='3'>3. Save data</a>
----

In [22]:
df_new_previous.to_csv(output_path + "previous_application_fraud.csv", index=False)
df_new.to_csv(output_path + "application_data_fraud.csv", index=False)
new_info_df.to_csv(output_path + "columns_description_fraud.csv", index=False)

---